<a href="https://colab.research.google.com/github/Aleksandras17/CNN-benign-or-malignant/blob/main/CNN_benign_malignant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip '/content/drive/MyDrive/machine learning data (1)/Preprocessing and augmentations.v26-noise-0-to-0.6.folder.zip' -d '/content/data'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
unzip:  cannot find or open /content/drive/MyDrive/machine learning data (1)/Preprocessing and augmentations.v26-noise-0-to-0.6.folder.zip, /content/drive/MyDrive/machine learning data (1)/Preprocessing and augmentations.v26-noise-0-to-0.6.folder.zip.zip or /content/drive/MyDrive/machine learning data (1)/Preprocessing and augmentations.v26-noise-0-to-0.6.folder.zip.ZIP.


In [ ]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

bsize=64

train_ds = datasets.ImageFolder("/content/drive/MyDrive/machine learning data/Preprocessing and augmentations.v26-noise-0-to-0.6.folder (Unzipped Files)/train", transform=transform)
train_loader = DataLoader(train_ds, batch_size=bsize, shuffle=True)

val_ds = datasets.ImageFolder("/content/drive/MyDrive/machine learning data/Preprocessing and augmentations.v26-noise-0-to-0.6.folder (Unzipped Files)/test", transform=transform)
val_loader = DataLoader(val_ds, batch_size=bsize, shuffle=False)

test_ds = datasets.ImageFolder("/content/drive/MyDrive/machine learning data/Preprocessing and augmentations.v26-noise-0-to-0.6.folder (Unzipped Files)/valid", transform=transform)
test_loader = DataLoader(test_ds, batch_size=bsize, shuffle=False)










In [ ]:
#Building the model
from torch import nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding = 1)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(32, 32, kernel_size=5, padding=2)

    self.fun1 = nn.Linear(32*32*32, 128)
    self.fun2 = nn.Linear(128, 2)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))

    x = self.pool(F.relu(self.conv2(x)))

    x = x.view(x.size(0), -1)


    x = F.relu(self.fun1(x))
    x = self.fun2(x)

    return x

print(NeuralNetwork())



NeuralNetwork(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fun1): Linear(in_features=32768, out_features=128, bias=True)
  (fun2): Linear(in_features=128, out_features=2, bias=True)
)


In [ ]:
from tqdm import tqdm
import sys
import numpy as np
import torch






def epoch(dataloader, loss_fn, model, optimizer):
  model.train()
  total_loss = 0
  total_batches = 0
  with tqdm(dataloader, unit="batch", file=sys.stdout) as ep_tqdm:
    ep_tqdm.set_description("Train")

    for inputs, labels in ep_tqdm:
      inputs, labels = inputs.to(device), labels.to(device)
      predictions = model(inputs)
      batch_loss = loss_fn(predictions, labels)

      optimizer.zero_grad()
      batch_loss.backward()
      optimizer.step()


      total_loss +=batch_loss.item()
      total_batches+=1
      ep_tqdm.set_postfix(average_batch_loss = (total_loss/total_batches))



def evaluation(dataloader, model, loss):
  model.eval()

  total_loss = 0
  total_samples = 0
  total_correct = 0
  total_batches = 0
  with torch.no_grad():
    with tqdm(dataloader, unit="batch", file=sys.stdout)as ep_tqdm:
      for inputs, labels in ep_tqdm:
          inputs, labels = inputs.to(device), labels.to(device)
          predictions = model(inputs)

          total_loss += loss(predictions, labels)
          total_correct += (predictions.argmax(dim=1) == labels).type(torch.float).sum()
          total_samples += len(inputs)
          total_batches += 1

          ep_tqdm.set_postfix(average_batch_loss=(total_loss/total_batches).item(), accuracy=(total_correct/total_samples).item())






In [ ]:
import torch.optim as optim
epochs = 5


device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

model = NeuralNetwork().to(device)

loss= nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for e in range(epochs):
  epoch(train_loader, loss, model, optimizer)
  evaluation(val_loader, model, loss)


Using cuda device
100%|██████████| 5/5 [00:01<00:00,  3.36batch/s, accuracy=0.767, average_batch_loss=0.47]


In [ ]:
from sklearn.metrics import classification_report
import torch
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"




with torch.no_grad():
  predictions = []
  real = []
  for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    preds = model(inputs).argmax(dim=1).cpu()

    predictions.extend(preds)
    real.extend(labels.cpu())
  print(classification_report(real, predictions))

              precision    recall  f1-score   support

           0       0.83      0.82      0.83       299
           1       0.82      0.83      0.83       299

    accuracy                           0.83       598
   macro avg       0.83      0.83      0.83       598
weighted avg       0.83      0.83      0.83       598

